# <center>Generating a Similarity matrix</center>

##### Installing and importing all the required modules

In [2]:
%%capture
!pip install antspyx

In [ ]:
import ants
import numpy as np
import os
import pandas as pd
from tqdm import tqdm

import warnings
warnings.filterwarnings(action='ignore')

##### Loading the data

In [6]:
data_dir=os.getcwd()

files=[i for i in os.listdir(data_dir) if '.nii.gz' in i]
files.sort()

##### Generating the similarity matrix

In [9]:
file_to_ind={v:i for i,v in enumerate(files)}
file_to_ind

{'VHF_Kidney_L_High.nii.gz': 0,
 'VHF_Kidney_R_High.nii.gz': 1,
 'VHM_Kidney_L_High.nii.gz': 2,
 'VHM_Kidney_R_High.nii.gz': 3}

##### Similarity based on Mean Squares Difference in Intensities

In [10]:
target_image=ants.image_read('/content/VHF_Kidney_L_High.nii.gz')

In [11]:
sim_mat=np.zeros((len(file_to_ind),len(file_to_ind)))
sim_mat

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

In [13]:
for i in files:
    img1=ants.image_read(os.path.join(data_dir,i)).resample_image_to_target(target_image)
    for j in files:
        img2=ants.image_read(os.path.join(data_dir,j)).resample_image_to_target(target_image)
        temp=ants.image_similarity(img1,img2,metric_type='MeanSquares')
        sim_mat[file_to_ind[i],file_to_ind[j]]=abs(temp)
        print(f"Mean Square difference between {i[:-12].replace('_Kidney','')} & {j[:-12].replace('_Kidney','')} ==> {abs(temp)}")

Mean Square difference between VHF_L & VHF_L ==> 0.0
Mean Square difference between VHF_L & VHF_R ==> inf
Mean Square difference between VHF_L & VHM_L ==> inf
Mean Square difference between VHF_L & VHM_R ==> inf
Mean Square difference between VHF_R & VHF_L ==> inf
Mean Square difference between VHF_R & VHF_R ==> 0.0
Mean Square difference between VHF_R & VHM_L ==> 0.020688209682703018
Mean Square difference between VHF_R & VHM_R ==> 0.020659802481532097
Mean Square difference between VHM_L & VHF_L ==> inf
Mean Square difference between VHM_L & VHF_R ==> 0.020688209682703018
Mean Square difference between VHM_L & VHM_L ==> 0.0
Mean Square difference between VHM_L & VHM_R ==> 0.00021376354561652988
Mean Square difference between VHM_R & VHF_L ==> inf
Mean Square difference between VHM_R & VHF_R ==> 0.020659802481532097
Mean Square difference between VHM_R & VHM_L ==> 0.00021376354561652988
Mean Square difference between VHM_R & VHM_R ==> 0.0


In [14]:
sim_mat

array([[0.        ,        inf,        inf,        inf],
       [       inf, 0.        , 0.02068821, 0.0206598 ],
       [       inf, 0.02068821, 0.        , 0.00021376],
       [       inf, 0.0206598 , 0.00021376, 0.        ]])

In [15]:
print('Metric type: Mean Squares\n')
df=pd.DataFrame(sim_mat,index=[i[:-12].replace('_Kidney','') for i in file_to_ind.keys()],columns=[i[:-12].replace('_Kidney','') for i in file_to_ind.keys()])
df.head()

Metric type: Mean Squares



,VHF_L,VHF_R,VHM_L,VHM_R
VHF_L,0.0,inf,inf,inf
VHF_R,inf,0.000000,0.020688,0.020660
VHM_L,inf,0.020688,0.000000,0.000214
VHM_R,inf,0.020660,0.000214,0.000000


In [16]:
df.to_csv('similarity matrix_meansquares.csv')

##### Similarity based on Correlation

In [17]:
sim_mat=np.zeros((len(file_to_ind),len(file_to_ind)))
sim_mat

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

In [18]:
for i in files:
    img1=ants.image_read(os.path.join(data_dir,i)).resample_image_to_target(target_image)
    for j in files:
        img2=ants.image_read(os.path.join(data_dir,j)).resample_image_to_target(target_image)
        temp=ants.image_similarity(img1,img2,metric_type='Correlation')
        sim_mat[file_to_ind[i],file_to_ind[j]]=abs(temp)
        print(f"Similarity between {i[:-12].replace('_Kidney','')} & {j[:-12].replace('_Kidney','')} ==> {abs(temp)}")

Similarity between VHF_L & VHF_L ==> 1.0
Similarity between VHF_L & VHF_R ==> 3.5264458020378697e-10
Similarity between VHF_L & VHM_L ==> 2.6301888184782074e-12
Similarity between VHF_L & VHM_R ==> 2.1689054632151317e-12
Similarity between VHF_R & VHF_L ==> 3.5264458020378697e-10
Similarity between VHF_R & VHF_R ==> 1.0
Similarity between VHF_R & VHM_L ==> 8.353114026249386e-06
Similarity between VHF_R & VHM_R ==> 6.888142706884537e-06
Similarity between VHM_L & VHF_L ==> 2.6301888184782074e-12
Similarity between VHM_L & VHF_R ==> 8.353114026249386e-06
Similarity between VHM_L & VHM_L ==> 1.0
Similarity between VHM_L & VHM_R ==> 0.025107936933636665
Similarity between VHM_R & VHF_L ==> 2.1689054632151317e-12
Similarity between VHM_R & VHF_R ==> 6.888142706884537e-06
Similarity between VHM_R & VHM_L ==> 0.025107936933636665
Similarity between VHM_R & VHM_R ==> 1.0


In [19]:
sim_mat

array([[1.00000000e+00, 3.52644580e-10, 2.63018882e-12, 2.16890546e-12],
       [3.52644580e-10, 1.00000000e+00, 8.35311403e-06, 6.88814271e-06],
       [2.63018882e-12, 8.35311403e-06, 1.00000000e+00, 2.51079369e-02],
       [2.16890546e-12, 6.88814271e-06, 2.51079369e-02, 1.00000000e+00]])

In [20]:
print('Metric type: Correlation\n')
df=pd.DataFrame(sim_mat,index=[i[:-12].replace('_Kidney','') for i in file_to_ind.keys()],columns=[i[:-12].replace('_Kidney','') for i in file_to_ind.keys()])
df.head()

Metric type: Correlation



,VHF_L,VHF_R,VHM_L,VHM_R
VHF_L,1.000000e+00,3.526446e-10,2.630189e-12,2.168905e-12
VHF_R,3.526446e-10,1.000000e+00,8.353114e-06,6.888143e-06
VHM_L,2.630189e-12,8.353114e-06,1.000000e+00,2.510794e-02
VHM_R,2.168905e-12,6.888143e-06,2.510794e-02,1.000000e+00


In [21]:
df.to_csv('similarity matrix_correlation.csv')